In [ ]:
# Imports
import os
import cv2
import dlib
import imutils
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from time import sleep
from math import sqrt
from tqdm import tqdm
from imutils import face_utils
from sklearn.metrics import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from google.colab.patches import cv2_imshow

# from google.colab import drive
# drive.mount('/content/drive')

A lo largo de este notebook, se visualizan alguna de las pruebas realizadas para el desarrollo del sistema de identificación de rostros.

En este caso, se ven los resultados obtenidos utilizando a 2 personas para el entrenamiento.

Se probarán los siguientes clasificadores:
- Logistic Regression
- Multi-Layer Perceptron
- K Nearest Neighbors

En primer lugar, sacaremos toda la información relevante de cada uno de los rostros utilizando técnicas de Visión por Computador. Esto no permitirá generar un dataset con el que realizaremos las pruebas.

En la siguiente imagen podemos observar los 68 puntos que se pueden sacar de una imagen y, por lo tanto, las 2278 distancias posibles entre todos estos puntos.

In [ ]:
keanu = cv2.imread('data/keanu_grande.jpg')

p = "models/shape_predictor_68_face_landmarks.dat"
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(p)

# Almacenar caras
data = []

# Preprocesar frame y obtener caras detectadas
# frame = imutils.resize(keanu, width=640)

gray = cv2.cvtColor(keanu, cv2.COLOR_BGR2GRAY)

rects = detector(gray, 0)

for (i, rect) in enumerate(rects):

    shape = predictor(gray, rect)
    shape = face_utils.shape_to_np(shape)

    for i in range(0,68):
        for j in range(i+1,68):

            # frame = cv2.line(frame,[, ],[],color=(255,0,0), thickness=1)
            start_point = (int(shape[i][0]), int(shape[i][1]))

            end_point = (int(shape[j][0]),int(shape[j][1]))

            color = (0, 255, 0)
            keanu = cv2.line(keanu, start_point, end_point, color, 1)

cv2_imshow(keanu)
#cv2.imwrite('keanu_todos.jpg', keanu)

Cargamos los datos previamente introducidos en un csv en un DataFrame

In [1]:
# Crear cabeceras del Dataframe
headers = []
for i in range(0,68):
    for j in range(i+1,68):
        headers.append(f'{i}TO{j}')
# print(headers)

In [ ]:
# Cargar train
landmarks_train_df = pd.read_csv("path/to/train/csv", columns=headers)

X_train = landmarks_train_df.drop(['Etiqueta'], axis=1)
y_train = landmarks_train_df['Etiqueta']

In [ ]:
# Cargar test
landmarks_test_df = pd.read_csv("path/to/test/csv", columns=headers)

X_test = landmarks_train_df.drop(['Etiqueta'], axis=1)
y_test = landmarks_train_df['Etiqueta']

# PRUEBA 1: UTILIZANDO TODAS LAS DISTANCIAS

Para esta primera prueba utilizaremos todos los datos posibles para hacer la clasificación con intención de obtener unas métricas base.

In [2]:
###### Regresión Logística ######

rl_model = LogisticRegression(max_iter=10000)
rl_model.fit(X_train, y_train)
best_value_lr = rl_model.score(X_test, y_test)


#### Multi-layer Perceptron #####

mlp_model = MLPClassifier(
    hidden_layer_sizes=[20, 40],
    activation='relu',
    early_stopping=True,
    random_state=13,
    max_iter=10000,
    solver='adam',
    verbose=False
)
mlp_model.fit(X_train, y_train)
best_value_mlp = mlp_model.score(X_test, y_test)


###### K Nearst Neighbors #######

vecinos =  []
for k in range(1, 101):
    model = KNeighborsClassifier(k)
    model.fit(X_train, y_train)
    vecinos.append((k, model.score(X_test, y_test)))

# Accuracy máximo obtenido
best_value_knn = max(vecinos, key=lambda x:x[1])[1]

########## Resultados ###########

print(f"K Nearst Neighbors - accuracy: {best_value_knn}")
print(f"Logistic Regression - accuracy: {best_value_lr}")
print(f"Multi-layer Perceptron - accuracy: {best_value_mlp}")

# Optimización de K

print('\nK optimization')
plt.scatter(*zip(*[i for i in vecinos if vecinos.index(i) % 2 == 0]))
plt.axis((0, 100, 0.5, 1))
plt.xlabel('Valor de k', fontsize=15)
plt.ylabel('Accuracy', fontsize=15)
plt.show()

SyntaxError: ignored

A pesar de que se obtienen muy buenos resultados, realizar el cálculo de todas las distancias por cada rostro y cada frame resulta muy costoso. Además, el algoritmo k-NN tiene un coste adicional, ya que para hacer cada predicción tiene que calcular todas las distancias entre los registros para buscar los k más cercanos. Por lo que será necesario hacer una reducción de atributos para que el funcionamiento del sistema sea viable.

# PRUEBA 2: SELECCIONANDO MANUALMENTE ALGUNAS DISTANCIAS

Para comprobar si es viable realizar el cálculo de menos distancias para la identificación en tiempo real, realizaremos otra prueba escogiendo manualmente las imágenes del rostro que se observan en la siguiente imagen:

In [9]:
imagen = cv2.imread('data/keanu_grande.jpg')

# Cargar el detector de rostros de DLIB
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("/content/sample_data/shape_predictor_68_face_landmarks.dat")

# Detectar rostros en la imagen
imagen_rgb = cv2.cvtColor(imagen, cv2.COLOR_BGR2RGB)
rostros = detector(imagen_rgb)

init = [1, 36, 42, 27, 31, 17, 22, 48, 6, 8, 51]
fin = [15, 39, 45, 33, 35, 21, 26, 54, 10, 57, 33]

for rostro in rostros:
    landmarks = predictor(imagen_rgb, rostro)

    for p1, p2 in zip(init, fin):
        cv2.line(imagen, (landmarks.part(p1).x, landmarks.part(p1).y), (landmarks.part(p2).x, landmarks.part(p2).y), (0, 255, 255), 2)
        cv2.circle(imagen, (landmarks.part(p1).x, landmarks.part(p1).y), 2, (0, 0, 255), -1)
        cv2.circle(imagen, (landmarks.part(p2).x, landmarks.part(p2).y), 2, (0, 0, 255), -1)

cv2_imshow(imagen)

NameError: ignored

In [ ]:
# Obtener columnas deseadas
init = [1, 36, 42, 27, 31, 17, 22, 48, 6, 8, 51]
fin = [15, 39, 45, 33, 35, 21, 26, 54, 10, 57, 33]
new_colums = list()
for p1, p2 in zip(init, fin):
    new_colums.append(f'{p1}TO{p2}')

# Cargar nuevo train y test

#TODO COGER SOLO LAS NEW COLUMNS
X_train_prueba2 = landmarks_train_df.drop(['Etiqueta'], axis=1)
y_train_prueba2 = landmarks_train_df['Etiqueta']

X_test_prueba2 = landmarks_train_df.drop(['Etiqueta'], axis=1)
y_test_prueba2 = landmarks_train_df['Etiqueta']

In [ ]:
###### Regresión Logística ######

rl_model = LogisticRegression(max_iter=10000)
rl_model.fit(X_train_prueba2, y_train_prueba2)
best_value_lr = rl_model.score(X_test_prueba2, y_test_prueba2)


#### Multi-layer Perceptron #####

mlp_model = MLPClassifier(
    hidden_layer_sizes=[20, 40],
    activation='relu',
    early_stopping=True,
    +
    random_state=13,
    max_iter=10000,
    solver='adam',
    verbose=False
)
mlp_model.fit(X_train_prueba2, y_train_prueba2)
best_value_mlp = mlp_model.score(X_test_prueba2, y_test_prueba2)


###### K Nearst Neighbors #######

vecinos =  []
for k in range(1, 101):
    model = KNeighborsClassifier(k)
    model.fit(X_train_prueba2, y_train_prueba2)
    vecinos.append((k, model.score(X_test_prueba2, y_test_prueba2)))

# Accuracy máximo obtenido
best_value_knn = max(vecinos, key=lambda x:x[1])[1]

########## Resultados ###########

print(f"K Nearst Neighbors - accuracy: {best_value_knn}")
print(f"Logistic Regression - accuracy: {best_value_lr}")
print(f"Multi-layer Perceptron - accuracy: {best_value_mlp}")

# Optimización de K

print('\nK optimization')
plt.scatter(*zip(*[i for i in vecinos if vecinos.index(i) % 2 == 0]))
plt.axis((0, 100, 0.5, 1))
plt.xlabel('Valor de k', fontsize=15)
plt.ylabel('Accuracy', fontsize=15)
plt.show()

Con era de esperar, los resultados han empeorado ya que las distancias se han elegido sin ningún criterio específico. Con esta prueba hemos logrado comprobar que con estas distancias la identificación en tiempo real es viable por lo que nuestra tarea consistirá en obtener las distancias más relevantes.

# PRUEBA 3: UTILIZANDO DISTANCIAS CON MAYOR CORRELACIÓN

En esta prueba, utilizaremos como criterio se elección de distancias la correlación con la clase. Para ello, veremos la distribución de correlaciones de las 2278 distancias y aplicaremos un umbral mínimo que se deba cumplir para que una distancia sea relevante.

In [ ]:
# Calcular correlación
train_df_num = landmarks_train_df.copy()
mapeo = {'ALEX': 0, 'ENDIKA': 1}
train_df_num['Etiqueta'] = landmarks_train_df['Etiqueta'].map(mapeo)

correlacion = train_df_num.corr()['Etiqueta']

In [4]:
# Visualizar la distribución de las distancias según su correlación con la clase

# Font
plt.rcParams['font.family'] = 'Serif'

# Ajustar ancho y alto del gráfico
plt.figure(figsize=(10, 5))

# Número de barras del histograma, colores y título
plt.hist(correlacion, bins=120, edgecolor='#555555', alpha=0.95, color='#ffce93')
plt.title('Distribución de los atributos por correlación', fontsize=16)
# Eje X
plt.xlabel('Correlación', fontsize=12)
plt.xticks(np.linspace(-1, 1, num=11), fontsize=10)
# Eje Y
plt.yticks(fontsize=10)
plt.ylabel('Número de atributos', fontsize=12)
plt.grid(True)

# Agregar una línea vertical en el 0
plt.axvline(0, color='red', linestyle='dashed', linewidth=1, label=f'')
# plt.legend()

# Mostrar el gráfico
plt.show()

NameError: ignored

Como podemos observar que la gran mayoría de las distancias tiene una correlación entre 0 y 0.5 con la clase. Nuestra tarea consiste en descartar aquellas que tengas valores mínimos ya que no aportan ningún tipo de información relevante al modelo y, además, pueden introducir ruido.

In [ ]:
# Obtener columnas con correlación que superen el threshold
CORRELATION_THRESHOLD = 0.6

corr_df = pd.DataFrame(data=correlacion)
corr_df.reset_index(inplace=True)
corr_df = corr_df.rename({'index': 'Puntos', 'Etiqueta': 'Correlacion'}, axis=1)

corr_df
best_headers = []
for i in range(0,len(corr_df)):
    if abs(corr_df['Correlacion'][i]) > CORRELATION_THRESHOLD and corr_df['Puntos'][i]!='Etiqueta':
        best_headers.append(f"{corr_df['Puntos'][i]}")

print(f"New headers: {best_headers}")
print(f"Num headers: {len(best_headers)}")

# Cargar nuevo train y test

#TODO COGER SOLO LAS BEST COLUMNS
X_train_prueba3 = landmarks_train_df.drop(['Etiqueta'], axis=1)
y_train_prueba3 = landmarks_train_df['Etiqueta']

X_test_prueba3 = landmarks_train_df.drop(['Etiqueta'], axis=1)
y_test_prueba3 = landmarks_train_df['Etiqueta']

En la siguiente imagen podemos observar las únicas distancias que superan el umbral:

In [ ]:
imagen = cv2.imread('data/keanu_grande.jpg')

for pts in best_headers:
    i, j = pts.split('TO')


    start_point = (shape[int(i)][0], shape[int(i)][1])

    end_point = (int(shape[int(j)][0]),int(shape[int(j)][1]))

    color = (0, 255, 0)
    keanu_importantes = cv2.line(keanu_importantes, start_point, end_point, color, 1)

# cv2.imwrite('keanu_importantes.jpg', keanu_importantes)
cv2_imshow(imagen)

In [ ]:
###### Regresión Logística ######

rl_model = LogisticRegression(max_iter=10000)
rl_model.fit(X_train_prueba3, y_train_prueba3)
best_value_lr = rl_model.score(X_test_prueba3, y_test_prueba3)


#### Multi-layer Perceptron #####

mlp_model = MLPClassifier(
    hidden_layer_sizes=[20, 40],
    activation='relu',
    early_stopping=True,
    +
    random_state=13,
    max_iter=10000,
    solver='adam',
    verbose=False
)
mlp_model.fit(X_train_prueba3, y_train_prueba3)
best_value_mlp = mlp_model.score(X_test_prueba3, y_test_prueba3)


###### K Nearst Neighbors #######

vecinos =  []
for k in range(1, 101):
    model = KNeighborsClassifier(k)
    model.fit(X_train_prueba3, y_train_prueba3)
    vecinos.append((k, model.score(X_test_prueba3, y_test_prueba3)))

# Accuracy máximo obtenido
best_value_knn = max(vecinos, key=lambda x:x[1])[1]

########## Resultados ###########

print(f"K Nearst Neighbors - accuracy: {best_value_knn}")
print(f"Logistic Regression - accuracy: {best_value_lr}")
print(f"Multi-layer Perceptron - accuracy: {best_value_mlp}")

# Optimización de K

print('\nK optimization')
plt.scatter(*zip(*[i for i in vecinos if vecinos.index(i) % 2 == 0]))
plt.axis((0, 100, 0.5, 1))
plt.xlabel('Valor de k', fontsize=15)
plt.ylabel('Accuracy', fontsize=15)
plt.show()

Observamos que utilizando las distancias con mayor correlación se obtienen mejores métricas y además garantizamos que sea viable realizar la identificación en tiempo real.

# CONCLUSIONES
En estas pruebas se han utilizado 2 caras distintas para la optimización, pero al hacer pruebas con otros rostros veíamos que los resultados no eran igual de bueno. Esto se debe a que a medida que se añaden distintos rostros, la correlación de cada distancia puede variar.

Ejemplo: Comparando a 2 personas que tienen una nariz del mismo tamaño, la correlación de las medidas internas de la nariz será 0, ya que no aporta ninguna información para distinguir estos rostro. Al introducir una nueva persona en el sistema con una nariz considerablemente más grande o más pequeña, hace que estas distancias sean más relevantes para la distinción, por lo que su correlación aumenta.

Para asegurarnos siempre de que se utilizan las distancias más relevantes utilizamos columnas dinámicas, es decir, durante el periodo de entrenamiento, se calcula automáticamente qué distancias superan el threshold y únicamente estas se utilizan para el entrenamiento. De esta forma, obtendremos buenas métricas independientemente de tipo o número de rostros.

El código del sistema desarrollado se encuentra en GitHub:
https://github.com/EndikaEiros/Detecion_Facial